In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


<h3>Loading Data</h3>

In [ ]:

#Train data
train_df = pd.read_csv('train.csv')
# Test data
test_df = pd.read_csv('test.csv')

print(train_df.shape)
print(test_df.shape)

<li> Check data types</li>

In [20]:

data_types = train_df.dtypes.unique()
print('This dataset has {} types of data {}'.format(len(data_types), data_types))

This dataset has 3 types of data [dtype('int64') dtype('float64') dtype('O')]


In [21]:
# Object data
cat_columns = []
for column in train_df.columns:
    if train_df[column].dtype == np.dtype('O'):
        cat_columns.append(column)
        
print(cat_columns)

['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8']


<p>These columns are classified as KPI for month 6 7 8, we don't have much information for what it represents,
 for that reason we are going to exclude them from the dataset.</p>
<p>We'll exclude them for train_df and test_df</p>
 

In [22]:
# remove columns:
# let's remove them
train_df.drop(['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8'], axis=1, inplace=True)
train_df.drop(['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8'], axis=1, inplace=True)
train_df.drop('id', axis=1, inplace=True)
#print(train_df.shape)
test_df.drop(['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8'], axis=1, inplace=True)
test_df.drop(['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8'], axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)
#print(test_df.shape)


<p>The next steps are for:</p>
<ul>
    <li>Search for duplicated data and remove it</li>
    <li>Search for null values and check how many there are:</li>
        <ul>
            <li>If the amount of missing values > 30%, remove the columns.</li>
            <li>If less, we'll fill the values with the mean values for numerical features and mode for categorical features.</li>
        </ul>
</ul>

In [13]:
train_df.drop_duplicates(inplace=True)

# Checking for null values:
null_values = pd.DataFrame(100*(train_df.isnull().sum() / train_df.shape[0]), columns=['Percentage'])
# checking which columns have more then 30% of null values
features = null_values.loc[null_values['Percentage'] >= 30].index
print('Features with over 30% null values: ',features)
# remove these features from both datasets

train_df.drop(features, axis=1, inplace=True)
test_df.drop(features, axis=1, inplace=True)

print('Train Shape: {}'.format(train_df.shape))
print('Test Shape: {}'.format(test_df.shape))

Features with over 30% null values:  Index(['date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'max_rech_data_6', 'max_rech_data_7',
       'max_rech_data_8', 'count_rech_2g_6', 'count_rech_2g_7',
       'count_rech_2g_8', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'av_rech_amt_data_6', 'av_rech_amt_data_7',
       'av_rech_amt_data_8', 'arpu_3g_6', 'arpu_3g_7', 'arpu_3g_8',
       'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'night_pck_user_6',
       'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7',
       'fb_user_8'],
      dtype='object')
Train Shape: (69999, 135)
Test Shape: (30000, 134)


<p>If we recheck the null values again, we'll probably have the remain columns with null values that, should have a percentage of missing values, below 30% </p>
<p>Let's check it out, to verify if everything went as expected</p>

In [14]:
# Checking for null values:
null_values = pd.DataFrame(100*(train_df.isnull().sum() / train_df.shape[0]), columns=['Percentage'])
null_values.describe()


,Percentage
count,135.000000
mean,2.832718
std,2.137240
min,0.000000
25%,0.000000
50%,3.838626
75%,3.954342
max,5.290076


<p>As we can see we have features with a max of 5.29% of missing values. Everything went as expected</p>
<p>In the next step, as stated before, we are going to fill in the values</p>
<br>
<p>We'll use also the mode values and mean values from the train_df to fill in the values missing from the test data</p>

In [15]:
# fill in the values with mean and mode.

for column in train_df.columns:
    if train_df[column].dtype == np.dtype('float64'):
        train_df[column].fillna(int(train_df[column].mean()), inplace=True)
        if column != 'churn_probability':
            test_df[column].fillna(int(train_df[column].mean()), inplace=True)
    elif train_df[column].dtype == np.dtype('int64'):
        train_df[column].fillna(int(train_df[column].mean()), inplace=True)
        if column != 'churn_probability':
            test_df[column].fillna(int(train_df[column].mean()), inplace=True)
    elif train_df[column].dtype == np.dtype('O'):
        train_df[column].fillna(train_df[column].mode(), inplace=True)
        if column != 'churn_probability':
            test_df[column].fillna(train_df[column].mode(), inplace=True)


<p>After these initial steps, let's check the general statistics for the train dataset.</p>

In [16]:
train_df.describe()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,...,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.0,69999.0,69999.0,69999.0,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,...,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.147214,133.860104,132.926506,...,0.077730,0.081958,0.075344,0.081444,0.085487,1220.639709,68.108597,65.935830,60.07674,0.101887
std,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,293.972405,305.244309,303.534682,...,0.383189,0.381821,0.573003,0.634547,0.680035,952.426321,269.328659,267.899034,257.22681,0.302502
min,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,8.060000,7.260000,7.360000,...,0.000000,0.000000,0.000000,0.000000,0.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,37.730000,35.690000,36.790000,...,0.000000,0.000000,0.000000,0.000000,0.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,133.000000,133.000000,132.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,10752.560000,...,16.000000,16.000000,29.000000,33.000000,41.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


<p>As we can see we have several features with only zeros or only one value, which don't have any useful information for our model. For that reason let's remove them.</p>

In [17]:
unique_value_columns = []
for column in train_df.columns:
    if train_df[column].nunique() == 1:
        unique_value_columns.append(column)
        
train_df.drop(unique_value_columns, axis=1, inplace=True)
test_df.drop(unique_value_columns, axis=1, inplace=True)


print('Train Shape: {}'.format(train_df.shape))
print('Test Shape: {}'.format(test_df.shape))

Train Shape: (69999, 125)
Test Shape: (30000, 124)


<p>After cleaning the dataset, it is time to save into a .csv, for further use.</p>

In [18]:
train_df.to_csv('train_df_cleaned.csv', index=False)
test_df.to_csv('test_df_cleaned.csv', index=False)

In [47]:
# Devide the Dataset between x and y
x_train = train_df.drop('churn_probability', axis=1)
y_train = train_df['churn_probability']

In [48]:
x_train.columns

Index(['id', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'arpu_6', 'arpu_7', 'arpu_8', 'onnet_mou_6', 'onnet_mou_7',
       ...
       'monthly_3g_6', 'monthly_3g_7', 'monthly_3g_8', 'sachet_3g_6',
       'sachet_3g_7', 'sachet_3g_8', 'aon', 'aug_vbc_3g', 'jul_vbc_3g',
       'jun_vbc_3g'],
      dtype='object', length=135)

In [7]:
# check dataset Balance
print(f'We have:{100*y_train.sum() / y_train.count()} % on people who churn')

# the ratio is 1:100

We have:10.188716981671167 % on people who churn


In [5]:
# Scaling the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_train)

In [6]:
# PCA, in order to reduce the dataset dimension to make the fitting more easily. Our vector are very large, thus
# with this method we are going reduce the overall size.

pca = PCA()
x_pca = pca.fit_transform(x_scaled)

In [8]:
# let's consider a good acumulated variance explained to be used when 0.9
# find how many vectors we need
accumulated = 0
idx_final = 0
for idx in range(len(pca.explained_variance_ratio_)):
    accumulated += pca.explained_variance_ratio_[idx]
    if accumulated > 0.9:
        idx_final = idx
        print(accumulated, idx)
        break
        
# slice only up to de idx where we have >= 0.9
x_pca = x_pca[:, 0:idx_final]
x_pca.shape

0.902282380116558 52


(69999, 52)

In [41]:
train_pca = pd.DataFrame(x_pca)
train_pca.shape

(69999, 52)

In [43]:
y_train_ = pd.DataFrame(y_train)
y_train_.shape

(69999, 1)

In [47]:
train_pca = pd.concat([train_pca,y_train_], axis=1)
train_pca.shape

(69999, 53)

In [48]:
train_pca.to_csv('train_pca.csv')